In [1]:
import pandas as pd
df_train_path = "/kaggle/input/home-credit-big-data/df_train_output.parquet"
df_train = pd.read_parquet(df_train_path)
df_train.shape

(1526659, 472)

In [2]:
import pandas as pd
df_test_path = "/kaggle/input/home-credit-big-data/df_test_output.parquet"
df_test = pd.read_parquet(df_test_path)
df_test.shape

(10, 471)

In [3]:
import pandas as pd
df_carcols_path = "/kaggle/input/home-credit-big-data/cat_cols_output.parquet"
cat_cols = pd.read_parquet(df_carcols_path)
cat_cols.shape

(72, 1)

In [4]:
df_train

,case_id,month,week_num,target,assignmentdate_238D,assignmentdate_4527235D,birthdate_574D,contractssum_5085716L,dateofbirth_337D,days120_123L,...,mean_pmts_dpd_1073P,mean_pmts_dpd_303P,mean_pmts_overdue_1140A,mean_pmts_overdue_1152A,var_pmts_dpd_1073P,var_pmts_dpd_303P,var_pmts_overdue_1140A,var_pmts_overdue_1152A,year,day
0,732422,201907,26,0,NaN,NaN,-12786.0,NaN,-12786.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,2019,4
1,993589,202006,76,0,NaN,NaN,NaN,51162.378906,-15022.0,1.0,...,2.384615,NaN,1147.043091,NaN,33.923077,NaN,7.839426e+06,NaN,2020,16
2,648093,201903,9,0,NaN,NaN,-10176.0,NaN,-10176.0,3.0,...,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000e+00,NaN,2019,11
3,1496283,201908,32,0,-608.0,NaN,-21902.0,NaN,-21902.0,2.0,...,0.000000,2.562500,0.000000,185.176575,0.000000,37.673386,0.000000e+00,3.373292e+05,2019,19
4,2665726,202002,57,0,NaN,14.0,NaN,NaN,-23325.0,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,2020,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526654,645109,201903,9,0,NaN,NaN,-16957.0,NaN,-16957.0,0.0,...,3.826087,NaN,212.614166,NaN,107.480194,NaN,7.763466e+05,NaN,2019,6
1526655,1524418,201909,35,0,NaN,NaN,NaN,NaN,-13794.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,2019,7
1526656,1557874,201909,38,0,NaN,NaN,NaN,NaN,-14547.0,1.0,...,0.000000,0.648649,0.000000,4.432433,0.000000,9.567568,0.000000e+00,3.642523e+02,2019,29
1526657,2631629,201910,41,0,NaN,14.0,NaN,NaN,-22689.0,0.0,...,0.878049,7.064516,2.653659,1469.058105,17.209757,131.729034,1.407500e+02,4.722832e+06,2019,15


In [5]:
from sklearn.model_selection import StratifiedGroupKFold
y = df_train["target"].copy()
weeks = df_train["week_num"].copy()
df_train= df_train.copy().drop(columns=["target", "case_id", "week_num"])
cv = StratifiedGroupKFold(n_splits=5, shuffle=False)
if isinstance(cat_cols, pd.DataFrame):
    cat_cols = cat_cols.iloc[:, 0].tolist()
elif isinstance(cat_cols, pd.Series):
    cat_cols = cat_cols.tolist()
df_train[cat_cols] = df_train[cat_cols].astype(str)
df_test[cat_cols] = df_test[cat_cols].astype(str)

In [6]:
if df_train is None:
    raise ValueError("df_train 未正确加载或定义。")


In [7]:
import pandas as pd
import numpy as np
import optuna
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier, early_stopping, log_evaluation
from sklearn.model_selection import StratifiedGroupKFold

# 将 cat_cols 列转换为列表
cat_cols_list = cat_cols  # 如果 cat_cols 已经是列表，可以直接使用
df_train_orig = df_train.copy()
df_train_orig['week_num'] = weeks
# 获取一次分割的训练和验证集索引
idx_train, idx_valid = next(cv.split(df_train_orig, y, groups=weeks))
X_train, y_train = df_train_orig.iloc[idx_train], y.iloc[idx_train]
X_valid, y_valid = df_train_orig.iloc[idx_valid], y.iloc[idx_valid]

# 确保类别列是分类类型
X_train[cat_cols_list] = X_train[cat_cols_list].astype("category")
X_valid[cat_cols_list] = X_valid[cat_cols_list].astype("category")

# 检查训练集和验证集中的类别分布
unique_y_train = np.unique(y_train)
unique_y_valid = np.unique(y_valid)
print(f'Train classes: {unique_y_train}, Validation classes: {unique_y_valid}')
if len(unique_y_train) < 2 or len(unique_y_valid) < 2:
    print("Only one class present in y_train or y_valid. Exiting.")
else:
    def stability_metric(y_true, y_pred):
        """
        Custom metric for model optimization during training
        """
        weeks_to_score = X_valid['week_num'].reset_index(drop=True)
        gini_in_time = []

        for week in weeks_to_score.unique():
            week_idx = weeks_to_score.eq(week)
            if len(np.unique(y_true[week_idx])) < 2:
                continue  # 跳过只有一个类别的情况
            gini = np.array(2 * roc_auc_score(y_true[week_idx], y_pred[week_idx]) - 1)
            gini_in_time.append(gini)
        w_fallingrate = 88.0
        w_resstd = -0.5
        x = np.arange(len(gini_in_time))
        y = np.array(gini_in_time)
        a, b = np.polyfit(x, y, 1)
        y_hat = a * x + b
        residuals = y - y_hat
        res_std = np.std(residuals)
        avg_gini = np.mean(y)
        stability_score = avg_gini + w_fallingrate * min(0, a) + w_resstd * res_std
        is_higher_better = True

        return 'stability_score', stability_score, is_higher_better

    def lgbm_objective(trial):
        """
        LGBMClassifier parameters search
        """
        # Target ratio for unbalanced data
        y_ratio = np.sum(y_train == 0) / np.sum(y_train == 1)

        params = {
            'boosting_type': 'gbdt',
            'colsample_bynode': 0.8,
            'colsample_bytree': 0.8,
            'device': 'gpu',
            'extra_trees': True,
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0),  # l1_regularization
            'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 10),  # l2_regularization
            'max_depth': trial.suggest_int('max_depth', 10, 20),
            'metric': 'auc',
            'n_estimators': 2000,
            'num_leaves': trial.suggest_int('num_leaves', 30, 150),
            'objective': 'binary',
            'random_state': 42,
            'verbose': -1,
            'scale_pos_weight': y_ratio,
        }
        model = LGBMClassifier(**params)

        # 再次检查类分布
        unique_y_train = np.unique(y_train)
        unique_y_valid = np.unique(y_valid)
        if len(unique_y_train) < 2 or len(unique_y_valid) < 2:
            raise optuna.TrialPruned("Skipping this trial due to only one class present in y_train or y_valid.")

        model.fit(X_train, y_train,
                  eval_set=[(X_valid, y_valid)],
                  eval_metric=stability_metric,
                  callbacks=[log_evaluation(100), early_stopping(100)],
                  )
        y_pred = model.predict_proba(X_valid)[:, 1]
        _, stability_score, _ = stability_metric(np.array(y_valid), np.array(y_pred))

        return stability_score

    # 执行 Optuna 优化
    sampler = optuna.samplers.TPESampler(multivariate=True)
    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(lgbm_objective, n_trials=30)

    # Show best results
    trial = study.best_trial

    print('Number of finished trials: ', len(study.trials))
    print('Best trial:')
    print('Value:', trial.value)
    print('Params:')

    for key, value in trial.params.items():
        print('{}: {}'.format(key, value))

    # 保存所有 trial 的结果到 CSV 文件中
    results = []
    for trial in study.trials:
        trial_result = trial.params
        trial_result['value'] = trial.value
        results.append(trial_result)
    df_results = pd.DataFrame(results)

/tmp/ipykernel_34/2425759029.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[cat_cols_list] = X_train[cat_cols_list].astype("category")
/tmp/ipykernel_34/2425759029.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[cat_cols_list] = X_valid[cat_cols_list].astype("category")
/opt/conda/lib/python3.10/site-packages/optuna/samplers/_tpe/sampler.py:319: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2024

Train classes: [0 1], Validation classes: [0 1]


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.842246	valid_0's stability_score: 0.668354
[200]	valid_0's auc: 0.845719	valid_0's stability_score: 0.676947
[300]	valid_0's auc: 0.845497	valid_0's stability_score: 0.677282
Early stopping, best iteration is:
[257]	valid_0's auc: 0.845915	valid_0's stability_score: 0.677939


[I 2024-05-25 14:06:26,132] Trial 0 finished with value: 0.6779386699236017 and parameters: {'learning_rate': 0.09935564821075346, 'reg_alpha': 0.4100440184188568, 'reg_lambda': 1.489585626358459, 'max_depth': 12, 'num_leaves': 108}. Best is trial 0 with value: 0.6779386699236017.


...

Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.828411	valid_0's stability_score: 0.552643
[200]	valid_0's auc: 0.837386	valid_0's stability_score: 0.606087
[300]	valid_0's auc: 0.843208	valid_0's stability_score: 0.643605
[400]	valid_0's auc: 0.846861	valid_0's stability_score: 0.667218
[500]	valid_0's auc: 0.849314	valid_0's stability_score: 0.67856
[600]	valid_0's auc: 0.851008	valid_0's stability_score: 0.683356
[700]	valid_0's auc: 0.852261	valid_0's stability_score: 0.687838
[800]	valid_0's auc: 0.85318	valid_0's stability_score: 0.689814
[900]	valid_0's auc: 0.853777	valid_0's stability_score: 0.69108
[1000]	valid_0's auc: 0.854282	valid_0's stability_score: 0.692049
[1100]	valid_0's auc: 0.854627	valid_0's stability_score: 0.692716
[1200]	valid_0's auc: 0.854863	valid_0's stability_score: 0.693299
[1300]	valid_0's auc: 0.854998	valid_0's stability_score: 0.693586
[1400]	valid_0's auc: 0.855106	valid_0's stability_score: 0.693786
[1500]	valid

[I 2024-05-25 20:36:11,558] Trial 34 finished with value: 0.6944673153618931 and parameters: {'learning_rate': 0.013722917978115104, 'reg_alpha': 0.6180167870934851, 'reg_lambda': 4.204636527779867, 'max_depth': 15, 'num_leaves': 104}. Best is trial 25 with value: 0.6967073494166182.


Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.844541	valid_0's stability_score: 0.673968
[200]	valid_0's auc: 0.849591	valid_0's stability_score: 0.685446
[300]	valid_0's auc: 0.849914	valid_0's stability_score: 0.685173
Early stopping, best iteration is:
[209]	valid_0's auc: 0.849705	valid_0's stability_score: 0.68565


[I 2024-05-25 20:39:52,330] Trial 35 finished with value: 0.6856502097831011 and parameters: {'learning_rate': 0.07907473285311206, 'reg_alpha': 0.36898252158705047, 'reg_lambda': 6.596623650513459, 'max_depth': 20, 'num_leaves': 100}. Best is trial 25 with value: 0.6967073494166182.
[LightGBM] [Fatal] Check failed: (best_split_info.left_count) > (0) at /usr/local/src/LightGBM/lightgbm-python/src/treelearner/serial_tree_learner.cpp, line 846 .

[W 2024-05-25 20:40:28,984] Trial 36 failed with parameters: {'learning_rate': 0.02619588994336271, 'reg_alpha': 0.5103244166486967, 'reg_lambda': 3.330896778972292, 'max_depth': 19, 'num_leaves': 138} because of the following error: LightGBMError('Check failed: (best_split_info.left_count) > (0) at /usr/local/src/LightGBM/lightgbm-python/src/treelearner/serial_tree_learner.cpp, line 846 .\n').
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_value

LightGBMError: Check failed: (best_split_info.left_count) > (0) at /usr/local/src/LightGBM/lightgbm-python/src/treelearner/serial_tree_learner.cpp, line 846 .


In [ ]:
params1 = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "n_estimators": 2000,  
    "colsample_bytree": 0.8,
    "colsample_bynode": 0.8,
    "verbose": -1,
    "random_state": 42,
    "extra_trees":True,
    "device": 'cpu', 
    "gpu_use_dp":True,
    "verbose": -1,
}
for key, value in params1.items():
    df_results[key] = value
params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 10,
    "learning_rate": 0.05,
    "n_estimators": 2000,
    "colsample_bytree": 0.8,
    "colsample_bynode": 0.8,
    "verbose": -1,
    "random_state": 42,
    "reg_alpha": 0.1,
    "reg_lambda": 10,
    "extra_trees": True,
    "gpu_use_dp":True,
    'num_leaves': 64,
    "device": 'cpu'
}

# 将参数字典转换为数据框
params_df = pd.DataFrame([params])

# 将参数行添加到 df_results 的最上面
df_results = pd.concat([params_df, df_results], ignore_index=True)

In [ ]:
filtered_results = pd.DataFrame(columns=df_results.columns)

# 保留第一行
filtered_results = pd.concat([filtered_results, df_results.iloc[[0]]], ignore_index=True)

# 遍历从第二行开始的每一行
for i in range(1, len(df_results)):
    current_value = df_results.loc[i, 'value']
    # 检查当前行的value是否在接下来的三行中没有被超过
    if i + 3 < len(df_results):
        if all(current_value > df_results.loc[i + j, 'value'] for j in range(1, 4)):
            filtered_results = pd.concat([filtered_results, df_results.iloc[[i]]], ignore_index=True)
    else:
        # 最后不满三行的情况
        if all(current_value > df_results.loc[i + j, 'value'] for j in range(1, len(df_results) - i)):
            filtered_results = pd.concat([filtered_results, df_results.iloc[[i]]], ignore_index=True)

# 检查结果
df_results=filtered_results[filtered_results['value']>0.5]

In [ ]:
df_results.to_parquet('df_results_output.parquet', index=False)

In [ ]:
best_params = []
for i in range(len(df_results)):
    params_comb = df_results.iloc[i].drop('value').to_dict()
    # 确保 max_depth 和 num_leaves 是整数
    params_comb['max_depth'] = int(params_comb['max_depth'])
    params_comb['num_leaves'] = int(params_comb['num_leaves'])
    best_params.append(params_comb)

# 输出最佳参数组合
print("Best Params Combinations:")
for params in best_params:
    print(params)
    print('---')

# 使用 StratifiedGroupKFold 进行交叉验证
skf = StratifiedGroupKFold(n_splits=2)

# 评估所有最佳参数组合的表现，选择表现最好的五个
evaluated_params = []

for params in best_params:
    cv_scores_lgb = []
    for idx_train, idx_valid in skf.split(df_train_orig, y, groups=weeks):
        X_train, y_train = df_train_orig.iloc[idx_train], y.iloc[idx_train]
        X_valid, y_valid = df_train_orig.iloc[idx_valid], y.iloc[idx_valid]

        # 确保类别列是分类类型
        X_train[cat_cols_list] = X_train[cat_cols_list].astype("category")
        X_valid[cat_cols_list] = X_valid[cat_cols_list].astype("category")
        
        model = LGBMClassifier(**params)
        model.fit(X_train, y_train,
                  eval_set=[(X_valid, y_valid)],
                  eval_metric='auc',
                  callbacks=[log_evaluation(100), early_stopping(100)])
        y_pred_valid = model.predict_proba(X_valid)[:, 1]
        auc_score = roc_auc_score(y_valid, y_pred_valid)
        cv_scores_lgb.append(auc_score)
    
    mean_auc = np.mean(cv_scores_lgb)
    evaluated_params.append((params, mean_auc))

# 选择最佳的五个参数组合
evaluated_params.sort(key=lambda x: x[1], reverse=True)
top_five_params = evaluated_params[:5]

for idx, (params, auc_score) in enumerate(top_five_params):
    print(f'Rank {idx+1} Average AUC Score: {auc_score}')
    print('Params:')
    for key, value in params.items():
        print('{}: {}'.format(key, value))
    print('---')

In [ ]:
import pandas as pd
import numpy as np
import gc
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.base import BaseEstimator, ClassifierMixin
import lightgbm as lgb

# 假设已经加载了原始数据 df_train_orig 和 df_test
# 例如：df_train_orig = pd.read_csv('train.csv')
# df_test = pd.read_csv('test.csv')

# 删除训练和测试数据中的 'week_num' 列
df_train = df_train_orig.drop(columns=['week_num'])
df_test = df_test.drop(columns=['week_num'])

# 定义特征
train_features = [col for col in df_train.columns if col not in ['target', 'weeks']]
test_features = [col for col in df_test.columns if col not in ['weeks', 'case_id']]

# 定义类别列
df_train[cat_cols] = df_train[cat_cols].astype("category")
df_test[cat_cols] = df_test[cat_cols].astype("category")

# 假设 top_five_params 包含了五个最优参数集


# 训练 LightGBM 模型
fitted_models_lgb = []
cv_scores_lgb = []

cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)  # 改为三折交叉验证
X = df_train[train_features]

for idx, (params, auc_score) in enumerate(top_five_params):
    for idx_train, idx_valid in cv.split(X, y, groups=weeks):
        X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
        X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]

        # 确保类别列是分类类型
        X_train[cat_cols] = X_train[cat_cols].astype("category")
        X_valid[cat_cols] = X_valid[cat_cols].astype("category")

        model = lgb.LGBMClassifier(**params)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_valid, y_valid)],
            eval_metric='auc',
            callbacks=[lgb.log_evaluation(50), lgb.early_stopping(50)],
        )
        fitted_models_lgb.append(model)

        y_pred_valid = model.predict_proba(X_valid)[:, 1]
        auc_score = roc_auc_score(y_valid, y_pred_valid)
        cv_scores_lgb.append(auc_score)

# 输出 LightGBM 的 CV AUC 分数
print(f"\nCV AUC scores for LightGBM: {cv_scores_lgb}")
print(f"Maximum CV AUC score for LightGBM: {max(cv_scores_lgb)}\n")



In [ ]:
import joblib

# Save individual LightGBM models
for i, model in enumerate(fitted_models_lgb):
    joblib.dump(model, f'lightgbm_model_{i}.pkl')